## Drum Kit Sounds: Audio Classification

Dataset Source: https://www.kaggle.com/datasets/anubhavchhabra/drum-kit-sound-samples

#### Install Missing Libraries

In [19]:
%pip install datasets transformers librosa

Note: you may need to restart the kernel to use updated packages.


#### Import Necessary Libraries

In [20]:
import os, sys, random
os.environ['TOKENIZERS_PARALLELISM']='false'

import numpy as np

import datasets
from datasets import load_dataset, Audio, DatasetDict

import transformers
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from transformers import TrainingArguments, Trainer

import evaluate

from IPython.display import Audio, display

#### Display Library Versions

In [21]:
print("Python:".rjust(15), sys.version[0:6])
print("NumPy:".rjust(15), np.__version__)
print("Datasets:".rjust(15), datasets.__version__)
print("Transformers:".rjust(15), transformers.__version__)
print("Evaluate:".rjust(15), evaluate.__version__)

        Python: 3.9.12
         NumPy: 1.23.3
      Datasets: 2.8.0
  Transformers: 4.25.1
      Evaluate: 0.2.2


#### Ingest Dataset

In [22]:
audio_data = load_dataset("/Users/briandunn/Desktop/Vit_Image_Datasets/Audio Data/Drum Kit Sound Samples/drums", 
                          name="en-US", 
                          split="train")

print(len(audio_data))

A Jupyter Widget

Using custom data configuration drums-5275f1687a56f823
Found cached dataset audiofolder (/Users/briandunn/.cache/huggingface/datasets/audiofolder/drums-5275f1687a56f823/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


160


#### Split Dataset into Training & Testing Datasets

In [23]:
# Train Test split dataset
audio_data_split = audio_data.train_test_split(test_size=0.20)

ds = DatasetDict({
    'train' : audio_data_split['train'],
    'eval' : audio_data_split['test']
})

#### Information about Training & Testing Datasets

In [24]:
print("Training Dataset")
print("Training Dataset Info: ", ds['train'])
print("First Sample in Training Dataset", ds['train'][0])
print("Last Sample in Training Dataset", ds['train'][-1])
print("Unique Values in Label/Class: ", ds['train'].unique("label"))

print("\n\nEvaluation Dataset")
print("Evaluation Dataset Info: ", ds['eval'])
print("First Sample in Evaluation Dataset", ds['eval'][0])
print("Last Sample in Evaluation Dataset", ds['eval'][-1])
print("Unique Values in Label/Class: ", ds['eval'].unique("label"))

Training Dataset
Training Dataset Info:  Dataset({
    features: ['audio', 'label'],
    num_rows: 128
})
First Sample in Training Dataset {'audio': {'path': '/Users/briandunn/Desktop/Vit_Image_Datasets/Audio Data/Drum Kit Sound Samples/drums/toms/Tom Sample 25.wav', 'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.9073486e-06,
       1.6093254e-06, 1.4305115e-06], dtype=float32), 'sampling_rate': 44100}, 'label': 3}
Last Sample in Training Dataset {'audio': {'path': '/Users/briandunn/Desktop/Vit_Image_Datasets/Audio Data/Drum Kit Sound Samples/drums/snare/Snare Sample 15.wav', 'array': array([ 4.7063828e-04, -1.5287995e-03, -2.3927331e-02, ...,
       -1.9967556e-05, -1.8537045e-05, -1.6987324e-05], dtype=float32), 'sampling_rate': 44100}, 'label': 2}


A Jupyter Widget

Unique Values in Label/Class:  [3, 2, 1, 0]


Evaluation Dataset
Evaluation Dataset Info:  Dataset({
    features: ['audio', 'label'],
    num_rows: 32
})
First Sample in Evaluation Dataset {'audio': {'path': '/Users/briandunn/Desktop/Vit_Image_Datasets/Audio Data/Drum Kit Sound Samples/drums/toms/Tom Sample 33.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00013328,
       -0.00015301, -0.00014514], dtype=float32), 'sampling_rate': 44100}, 'label': 3}
Last Sample in Evaluation Dataset {'audio': {'path': '/Users/briandunn/Desktop/Vit_Image_Datasets/Audio Data/Drum Kit Sound Samples/drums/toms/Tom Sample 34.wav', 'array': array([ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  2.014637e-05,
        1.013279e-06, -1.758337e-05], dtype=float32), 'sampling_rate': 44100}, 'label': 3}


A Jupyter Widget

Unique Values in Label/Class:  [3, 1, 0, 2]


#### Create Dictionaries to Convert Labels Between Strings & Integers

In [25]:
labels = ds["train"].features["label"].names

print(labels)

label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

['kick', 'overheads', 'snare', 'toms']


#### Display Some Examples with Ability to Listen to Them

In [26]:
for _ in range(5):
    rand_idx = random.randint(0, len(ds["train"])-1)
    example = ds["train"][rand_idx]
    audio = example["audio"]
    
    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: toms
Shape: (88200,), sampling rate: 44100



Label: toms
Shape: (88200,), sampling rate: 44100



Label: toms
Shape: (88200,), sampling rate: 44100



Label: snare
Shape: (88200,), sampling rate: 44100



Label: kick
Shape: (88200,), sampling rate: 44100


#### Remember to Install git lfs & Enter HuggingFace Access Token

In [27]:
!git lfs install

# HuggingFace Access Token ...

Git LFS initialized.


#### Basic Values/Constants

In [28]:
MODEL_CKPT = "facebook/wav2vec2-base"
MODEL_NAME = MODEL_CKPT.split("/")[-1] + "-Drum_Kit_Sounds"

NUM_OF_EPOCHS = 12
LEARNING_RATE = 3e-5

BATCH_SIZE = 32
STRATEGY = "epoch"

#### Set Sample Rate

In [29]:
sampling_rate = ds["train"].features["audio"].sampling_rate
sampling_rate

#### Instantiate Instance of Feature Extractor

In [30]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_CKPT)

loading configuration file preprocessor_config.json from cache at /Users/briandunn/.cache/huggingface/hub/models--facebook--wav2vec2-base/snapshots/0b5b8e868dd84f03fd87d01f9c4ff0f080fecfe8/preprocessor_config.json
loading configuration file config.json from cache at /Users/briandunn/.cache/huggingface/hub/models--facebook--wav2vec2-base/snapshots/0b5b8e868dd84f03fd87d01f9c4ff0f080fecfe8/config.json
/Users/briandunn/Documents/nlpnn/nlp_ml/lib/python3.9/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply

#### Define function to Preprocess Data

In [31]:
def preprocess_function(examples):
    '''
    This function prepares the dataset for the transformer
    by applying the feature extractor to it (among other 
    processes).
    '''
    max_duration = 1.0 # seconds
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, 
                               sampling_rate=feature_extractor.sampling_rate, 
                               max_length=int(feature_extractor.sampling_rate * max_duration),
                               truncation=True)
    return inputs

encoded_audio = ds.map(preprocess_function, remove_columns="audio", batched=True)

A Jupyter Widget

A Jupyter Widget

#### Define Metrics Evaluation Function 

In [32]:
def compute_metrics(p):
    accuracy_metric = evaluate.load("accuracy")
    accuracy = accuracy_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)['accuracy']
    
    ### ------------------- F1 scores -------------------
    
    f1_score_metric = evaluate.load("f1")
    weighted_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["f1"]
    micro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')['f1']
    macro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["f1"]
    
    ### ------------------- recall -------------------
    
    recall_metric = evaluate.load("recall")
    weighted_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["recall"]
    micro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["recall"]
    macro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["recall"]
    
    ### ------------------- precision -------------------
    
    precision_metric = evaluate.load("precision")
    weighted_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["precision"]
    micro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["precision"]
    macro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["precision"]
    
    return {"accuracy" : accuracy, 
            "Weighted F1" : weighted_f1_score,
            "Micro F1" : micro_f1_score,
            "Macro F1" : macro_f1_score,
            "Weighted Recall" : weighted_recall,
            "Micro Recall" : micro_recall,
            "Macro Recall" : macro_recall,
            "Weighted Precision" : weighted_precision,
            "Micro Precision" : micro_precision,
            "Macro Precision" : macro_precision
            }

#### Instantiate Model

In [33]:
num_of_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(MODEL_CKPT, 
                                                        num_labels=num_of_labels, 
                                                        label2id=label2id,
                                                        id2label= id2label)

loading configuration file config.json from cache at /Users/briandunn/.cache/huggingface/hub/models--facebook--wav2vec2-base/snapshots/0b5b8e868dd84f03fd87d01f9c4ff0f080fecfe8/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "fe

#### Define Training Arguments

In [34]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy=STRATEGY,
    num_train_epochs=NUM_OF_EPOCHS,
    save_strategy=STRATEGY,
    logging_strategy=STRATEGY,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.10,
    load_best_model_at_end=True,
    logging_first_step=True,
    hub_private_repo=True,
    push_to_hub=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


#### Define Trainer

In [35]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = encoded_audio["train"],
    eval_dataset = encoded_audio["eval"],
    tokenizer = feature_extractor,
    compute_metrics = compute_metrics
)

/Users/briandunn/Documents/nlpnn/nlp_ml/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in '__init__': private. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
/Users/briandunn/Documents/nlpnn/Audio Projects/wav2vec2-base-Drum_Kit_Sounds is already a clone of https://huggingface.co/DunnBC22/wav2vec2-base-Drum_Kit_Sounds. Make sure you pull the latest changes with `repo.git_pull()`.


#### Train Model

In [36]:
trainer.train()

/Users/briandunn/Documents/nlpnn/nlp_ml/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 128
  Num Epochs = 12
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 48
  Number of trainable parameters = 94569604


A Jupyter Widget

/Users/briandunn/Documents/nlpnn/nlp_ml/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'loss': 1.3588, 'learning_rate': 6e-06, 'epoch': 0.25}


***** Running Evaluation *****
  Num examples = 32
  Batch size = 32


...

Saving model checkpoint to wav2vec2-base-Drum_Kit_Sounds/checkpoint-48
Configuration saved in wav2vec2-base-Drum_Kit_Sounds/checkpoint-48/config.json


{'eval_loss': 1.0886529684066772, 'eval_accuracy': 0.78125, 'eval_Weighted F1': 0.7691964285714286, 'eval_Micro F1': 0.78125, 'eval_Macro F1': 0.7845238095238096, 'eval_Weighted Recall': 0.78125, 'eval_Micro Recall': 0.78125, 'eval_Macro Recall': 0.81875, 'eval_Weighted Precision': 0.8716947115384616, 'eval_Micro Precision': 0.78125, 'eval_Macro Precision': 0.8533653846153846, 'eval_runtime': 16.2005, 'eval_samples_per_second': 1.975, 'eval_steps_per_second': 0.062, 'epoch': 12.0}


Model weights saved in wav2vec2-base-Drum_Kit_Sounds/checkpoint-48/pytorch_model.bin
Feature extractor saved in wav2vec2-base-Drum_Kit_Sounds/checkpoint-48/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from wav2vec2-base-Drum_Kit_Sounds/checkpoint-48 (score: 1.0886529684066772).


{'train_runtime': 2710.1217, 'train_samples_per_second': 0.567, 'train_steps_per_second': 0.018, 'train_loss': 1.1549896597862244, 'epoch': 12.0}


TrainOutput(global_step=48, training_loss=1.1549896597862244, metrics={'train_runtime': 2710.1217, 'train_samples_per_second': 0.567, 'train_steps_per_second': 0.018, 'train_loss': 1.1549896597862244, 'epoch': 12.0})

#### Evaluate Model

In [37]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 32
  Batch size = 32


A Jupyter Widget

{'eval_loss': 1.0886529684066772,
 'eval_accuracy': 0.78125,
 'eval_Weighted F1': 0.7691964285714286,
 'eval_Micro F1': 0.78125,
 'eval_Macro F1': 0.7845238095238096,
 'eval_Weighted Recall': 0.78125,
 'eval_Micro Recall': 0.78125,
 'eval_Macro Recall': 0.81875,
 'eval_Weighted Precision': 0.8716947115384616,
 'eval_Micro Precision': 0.78125,
 'eval_Macro Precision': 0.8533653846153846,
 'eval_runtime': 16.1801,
 'eval_samples_per_second': 1.978,
 'eval_steps_per_second': 0.062,
 'epoch': 12.0}

#### Push Model to Hub (My Profile!!!)

In [38]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-base-Drum_Kit_Sounds
Configuration saved in wav2vec2-base-Drum_Kit_Sounds/config.json
Model weights saved in wav2vec2-base-Drum_Kit_Sounds/pytorch_model.bin
Feature extractor saved in wav2vec2-base-Drum_Kit_Sounds/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/DunnBC22/wav2vec2-base-Drum_Kit_Sounds
   32a9dd7..c37fc6e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'audiofolder', 'type': 'audiofolder', 'config': 'drums', 'split': 'train', 'args': 'drums'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.78125}]}
To https://huggingface.co/DunnBC22/wav2vec2-base-Drum_Kit_Sounds
   c37fc6e..60efa5c  main -> main



'https://huggingface.co/DunnBC22/wav2vec2-base-Drum_Kit_Sounds/commit/c37fc6e2112e1e7564f8a0ce66024e60a4ddff94'

### Notes & Other Takeaways
****
- I was not expecting this project to yield metrics this good. That said, it still is not up to where I think it should be to go further with it. 
- Including more samples would definitely improve this model.
****